In [5]:
import streamlit as st  # to render the user interface.
from langchain_community.llms import Ollama  # to use Ollama llms in langchain
from langchain_core.prompts import ChatPromptTemplate  # crafts prompts for our llm
from langchain_community.chat_message_histories import\
    StreamlitChatMessageHistory  # stores message history
from langchain_core.tools import tool  # tools for our llm
# to describe tools as a string
from langchain.tools.render import render_text_description
# ensure JSON input for tools
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter  # to retrieve specific items in our chain.

In [6]:
model = Ollama(model='mistral:instruct')

In [7]:
@tool
def add(first: int, second: int) -> int:
    "Add two integers."
    return first + second

In [8]:
print(add.name)
print(add.description)
print(add.args)

add
Add two integers.
{'first': {'title': 'First', 'type': 'integer'}, 'second': {'title': 'Second', 'type': 'integer'}}


In [9]:
add.invoke({'first': 3, 'second': 6})

9

In [10]:
@tool
def multiply(first: int, second: int) -> int:
    """Multiply two integers together."""
    return first * second

In [11]:
@tool
def converse(input: str) -> str:
    "Provide a natural language response using the user input."
    return model.invoke(input)

In [12]:
tools = [add, multiply, converse]
rendered_tools = render_text_description(tools)
print(rendered_tools)

add(first: int, second: int) -> int - Add two integers.
multiply(first: int, second: int) -> int - Multiply two integers together.
converse(input: str) -> str - Provide a natural language response using the user input.


In [13]:
system_prompt = f"""You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [15]:
chain = prompt | model | JsonOutputParser()

In [16]:
chain.invoke({'input': 'What is 3 times 23'})

{'name': 'multiply', 'arguments': {'first': 3, 'second': 23}}

In [17]:
chain.invoke({'input': 'How are you today?'})

{'name': 'converse', 'arguments': {'input': 'How are you today?'}}

In [19]:
# Define a function which returns the chosen tool
# to be run as part of the chain.
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [20]:
chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({'input': 'What is 3 times 23'})

69

# Streamlist chat history


In [21]:
# Set up message history.
msgs = StreamlitChatMessageHistory(key="langchain_messages")
if len(msgs.messages) == 0:
    msgs.add_ai_message(
        "I can add, multiply, or just chat! How can I help you?")

st.title("Chatbot with tools")

# Render the chat history.
for msg in msgs.messages:
    st.chat_message(msg.type).write(msg.content)


# React to user input
if input := st.chat_input("What is up?"):
    # Display user input and save to message history.
    st.chat_message("user").write(input)
    msgs.add_user_message(input)
    # Invoke chain to get reponse.
    response = chain.invoke({'input': input})
    # Display AI assistant response and save to message history.
    st.chat_message("assistant").write(str(response))
    msgs.add_ai_message(response)

2024-07-02 20:29:46.776 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-07-02 20:29:46.795 
  command:

    streamlit run /Users/avatsaev/miniconda3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
